# NLIL

In [1]:
from collections import defaultdict
import re

import pandas as pd

## 簡單分幾個column

In [2]:
raw = pd.read_json('data/news.json')

In [3]:
data = defaultdict(list)
columns = 'title time from content'.split()
pattern = ('(?P<title>.*|)'
           ' - 中時電子報\s*'
           '(?P<time>20[0-1][0-9]年[0-1][0-9]月[0-3][0-9]日 [0-2][0-9]:[0-5][0-9])'
           '(中國時報|中時及時) '
           '(?P<from>[^\n]*)'
           '\s*'
           '(?P<content>.*$)')
pattern = re.compile(pattern, flags=re.DOTALL)
for i, news in enumerate(raw.news, 1):
    match = pattern.search(news)
    if match is None: continue
    for col in columns:
        data[col].append(match.group(col))
data = pd.DataFrame(data)[columns]

## Split

- 根據有沒有出現法、律、條例、通則分成 `implicit` 和 `explicit`
- 應該要先針對 `explicit` 做點事 QAQ

In [8]:
related_words = {'法', '律', '條例', '通則'}
explicit = data.content.map(lambda s: any(word in s for word in related_words))
name = 'implicit', 'explicit'
for i, group in data.groupby(explicit):
    group.to_json('data/%s.jsonl'%name[i], lines=True, orient='records', force_ascii=False)

## 斷詞

- 先用 Jseg 斷一個， ckip 好麻煩

In [1]:
from jseg.jieba import Jieba
segmenter = Jieba()

DEBUG:jseg.jieba:loading default dictionary


In [4]:
import json
def pos_piece(piece, **arg):
    if piece['title']:
        title = segmenter.seg(piece['title'], **arg).raw
    else:
        title = ''
    content = segmenter.seg(piece['content'], **arg).raw
    piece['pos_title'] = title
    piece['pos_content'] = content
    return piece

In [3]:
with open('data/explicit.jsonl', 'r') as f:
    data = [json.loads(piece) for piece in f]

In [4]:
for i, piece in enumerate(data, 1):
    pos_piece(piece, pos=True)
    print('%5d/%5d'%(i, len(data)), end='\r')
print(data[0])

{'title': '', 'time': '2010年10月24日 01:30', 'content': '七年前，年僅九歲及八歲的板橋市某國小劉姓姊妹，遭陳昆明殘忍殺害。案發後，他企圖以精神病脫罪，但更一審送鑑定結果是「無精神病」。無奈的是，原本獲判無期徒刑的陳嫌，兩度更審讓刑期縮減為十二年，減刑出獄的結果，卻是葬送另一條寶貴生命。\n九十二年四月十六日，劉姓姊妹放學後，從此人間蒸發。警方成立「○四一六專案小組」，但案情始終陷入膠著，直到死者父親回憶起諸多蛛絲馬跡，發現九十年間因店面樓上整修，臨時工陳昆明出入近兩個月，從而與他的小孩混得很熟，直到陳當兵才失去聯絡。警方靠著這條線索，加上鄰居在案發前見到陳昆明在劉家附近徘徊，鎖定他涉有重嫌。\n陳昆明案發前在劉家外等候兩姊妹，相約十六日放學後一同前往大賣場，兩姊妹上鉤後，他將兩人載往華中橋墩下小木屋，先騙姊姊喝下摻有ＦＭ２的飲料，待藥效發作，動手掐死姊姊；陳嫌接著再掐妹妹頸部，但妹妹踢打反抗、哀嚎，陳昆明隨手持磚塊重擊她頭部致死，事後將兩女童屍體丟入新店溪。\n女童雙屍案偵結後，檢方依殺人罪將陳昆明提起公訴並具體求處極刑，一、二審均判處無期徒刑，但上訴高院之後，陳昆明的委任律師請求再送專門治療精神病的桃園療養院鑑定，但高院未予理會並維持原判。案經上訴，最高法院認為，陳昆明的精神狀態確有詳查必要，乃發回更審。\n不料，高院更一審將陳昆明送桃園療養院鑑定，結果卻是「陳昆明並無精神病」，雖維持原判，但後續更審的結果，陳昆明先改判十六年，再改判十二年，最後搭上減刑的順風車，在去年七月出獄。\n原本應與世隔絕的陳昆明，受惠國內司法制度減刑出獄，但他心中的惡魔卻伺機而動。經過一年的計畫，從假藉應徵手段到分租單身公寓，陳昆明預謀犯案的慾念似乎從未間斷，而現行司法制度間接造成無辜的林女枉死，未來應更慎重斟酌量刑。\n(中國時報)\n', 'from': '顏玉龍／新聞幕後', 'pos_title': '', 'pos_content': [('七', 'Ndabc'), ('年前', 'NN'), ('，', 'NN'), ('年僅', 'NN'), ('九', 'Neu'), ('歲', 'Nfg'), ('及', 'Caa'), ('八', 'Neu'), ('歲', 'Nfg'), ('的', 'DE'), ('板橋市',

In [5]:
with open('data/explicit_jseg.jsonl', 'w') as f:
    print(*(json.dumps(piece, ensure_ascii=False) for piece in data), sep='\n', file=f)

---

# CKIP segmenter

```python
from PyCCS import ckip
segmenter = ckip

with open('data/explicit.jsonl', 'r') as f:
    data = [json.loads(piece) for piece in f]
with open('data/explicit_ckip.jsonl', 'w') as f:
    for i, piece in enumerate(data, 1):
        pos_piece(piece)
        print(json.dumps(piece, ensure_ascii=False), file=f)
        print('%5d/%5d'%(i, len(data)), end='\r')
```

In [5]:
from PyCCS import ckip
segmenter = ckip

with open('data/implicit.jsonl', 'r') as f:
    data = [json.loads(piece) for piece in f]
with open('data/implicit_ckip.jsonl', 'w') as f:
    for i, piece in enumerate(data, 1):
        pos_piece(piece)
        print(json.dumps(piece, ensure_ascii=False), file=f)
        print('%5d/%5d'%(i, len(data)), end='\r')

---